<a href="https://colab.research.google.com/github/AniketGaudgaul/AI-ML-DL_Projects/blob/main/aces_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:

from PyPDF2 import PdfReader
import pdfplumber
import pandas as pd
import re

file = "/content/2 TE Sem I A.Y 22-23.pdf"

lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('............CONFIDENTIAL- FOR VERIFICATION AND RECORD ONLY AT COLLEGE, NOT FOR DISTRIBUTION.......................................'):
            x=[i for i in line.split('\n') if len(i)>1]
            if len(x)<1:
              continue
            if "PAGE" in x[0]:

              x.pop(0)
              x.pop(0)
              x.pop(0)
              x.pop(0)
            lines.append(x)


# A function to extract data from each entry

def extract_data(entry):
    # Initialize an empty dictionary for storing the extracted data
    data = {}

    # Extract name and mother's name
    name_line = entry[0]
    # print(name_line)

    data['name'] = name_line.split('NAME : ')[1].split(' MOTHER :')[0]
    data['mother_name'] = name_line.split('MOTHER : ')[1].split(' PRN :')[0]
    data['seat_number'] = name_line.split('SEAT NO.: ')[1].split(' NAME :')[0]
    data['prn'] = name_line.split('PRN :')[1].split(' CLG.:')[0]

    # print(data['name'])
    # print(data['mother_name'])
    # print(data['seat_number'])

    # print("entry-1",entry[-1])

    # Extract marks for each subject in sem 1
    result = {}
    sem_lines = entry[2:-1]

    # print(sem1_lines)

    sem = 1

    for line in sem_lines:
        if 'SEM.:1' in line:
            continue
        elif 'SEM.:2' in line:
            sem=2
            continue
        elif 'SGPA' in line:
            break
        else:
            tokens = line.split()
            tokens.reverse()
            # print(tokens)
            code = tokens[-1]
            course_name = ' '.join(tokens[13:-1])  # Extract course name

            # Extract insem marks if available
            if tokens[12] != '---' and 'AB' not in tokens[12]:
                insem_obtain, insem_total = map(int, tokens[12].split('/'))
            elif tokens[12]!='---' and 'AB' in tokens[12]:
                insem_obtain, insem_total = tokens[11].split('/')
                insem_total = int(insem_total)
            else:
                insem_obtain, insem_total = None, None

            # Extract ese marks if available
            if tokens[11] != '---' and 'AB' not in tokens[11] and '$' not in tokens[11] and '#' not in tokens[11]:
                ese_obtain, ese_total = map(int, tokens[11].split('/'))
            elif('AB' in tokens[11]):
                ese_obtain, ese_total = tokens[11].split('/')
                ese_total = int(ese_total)
            elif tokens[11] != '---' and '$' in tokens[11]:
                ese_obtain, ese_total = tokens[11].split('$/')
                ese_obtain = int(ese_obtain)
                ese_total = int(ese_total)
            elif tokens[11] != '---' and '#' in tokens[11]:
                ese_obtain, ese_total = tokens[11].split('#/')
                ese_obtain = int(ese_obtain)
                ese_total = int(ese_total)

            else:
                ese_obtain, ese_total = None, None

            # Extract total marks if available
            if tokens[10] != '---' and 'AB' not in tokens[10] and '#' not in tokens[10]:
                total_obtain, total_total = map(int, tokens[10].split('/'))
            elif('AB' in tokens[10]):
                total_obtain, total_total = tokens[10].split('/')
                total_total = int(total_total)
            elif(tokens[10] != '---' and '#' in tokens[10]):
                total_obtain, total_total = map(int, tokens[10].split('#/'))
            else:
                total_obtain, total_total = None, None

            # Extract term work marks if available
            if tokens[9] != '---' and 'PP' not in tokens[9] and 'AB' not in tokens[9] and 'AC' not in tokens[9]:
                tw_obtain, tw_total = map(int, tokens[9].split('/'))
            elif tokens[9] != '---' and 'PP' in tokens[9]:
                tw_obtain = tw_total = 'PP'
            elif tokens[9] != '---' and 'AC' in tokens[9]:
                tw_obtain = tw_total = 'AC'
            elif tokens[9] != '---' and 'AB' in tokens[9]:
                tw_obtain, tw_total = tokens[9].split('/')
                tw_total = int(tw_total)
            else:
                tw_obtain, tw_total = None, None

            # Extract practical marks if available
            if tokens[8] != '---' and 'AB' not in tokens[8] and '#' not in tokens[8]:
                pr_obtain, pr_total = map(int, tokens[8].split('/'))
            elif('AB' in tokens[8]):
                pr_obtain, pr_total = tokens[8].split('/')
                pr_total = int(pr_total)
            elif('#' in tokens[8]):
                pr_obtain, pr_total = tokens[8].split('#/')
                pr_obtain = int(pr_obtain)
                pr_total = int(pr_total)
            else:
                pr_obtain, pr_total = None, None

            # Extract oral marks if available
            if tokens[7] != '---':
                or_obtain, or_total = map(int, tokens[7].split('/'))
            else:
                or_obtain, or_total = None, None

            # Extract percentage and credits
            if(tokens[6]=='AC' or tokens[6] =="PP" or tokens[6] =='FF'):
              percent = tokens[6]
            else:
              percent = int(tokens[6])

            credits = int(tokens[5])

            # Extract grade, grade points, and credit points
            grade = tokens[4]
            grade_points = float(tokens[3])
            credit_points = float(tokens[2])

            # Extract pass and remanded status
            if(tokens[1]!='---'):
                pass_or_remanded_status = tokens[1]
            else:
                pass_or_remanded_status = None

            # Extract order of appearance and seat number
            if(tokens[0]!='---'):
                order_of_appearance = tokens[0]
            else:
                order_of_appearance = None

            # seat_number = tokens[-15]

            # reversing words in a given string
            s = course_name.split()[::-1]
            l = []
            for i in s:
                # apending reversed words to l
                l.append(i)
            # printing reverse words
            cn = " ".join(l)

            if(sem==1):
                semester = "sem1"
            else:
                semester = "sem2"

            result[code] = {'code': code, 'course_name': cn, 'insem_obtain': insem_obtain,
            'insem_total': insem_total, 'ese_obtain': ese_obtain, 'ese_total': ese_total,
            'total_obtain': total_obtain, 'total_total': total_total, 'tw_obtain': tw_obtain,
            'tw_total': tw_total, 'pr_obtain': pr_obtain, 'pr_total': pr_total,
            'or_obtain': or_obtain, 'or_total': or_total, 'percent': percent,
            'credits': credits, 'grade': grade, 'grade_points': grade_points,
            'credit_points': credit_points, 'pass_or_remanded_status': pass_or_remanded_status,
            'order_of_appearance': order_of_appearance, 'semester':semester}


    # print(sem1)
    data['result'] = result
    # print(data['sem1'])

    final_line = entry[-1]
    # print('final line', final_line)

    if 'SGPA1' in final_line:
        data['SGPA'] = final_line.split('SGPA1 : ')[1].split(',')[0]
    else:
        data['SGPA'] = final_line.split('SGPA : ')[1].split(',')[0]
    if data['SGPA']!='--' :
      float(data['SGPA'])

    data['cred_earned'] = float(final_line.split('TOTAL CREDITS EARNED : ')[1].split(' ')[0])
    # print(data['SGPA'])
    # print(data['cred_earned'])


    return data

# Iterate through each entry in the list and extract the data
students_data = []
for idx,entry in enumerate(lines):
    # print(entry)
    data = extract_data(entry)
    # print(idx)
    students_data.append(data)
    print(students_data[idx])
    print("\n")


In [ ]:
print(students_data[24])

{'name': 'GAUDGAUL ANIKET SANJEEV', 'mother_name': 'LATA GAUDGAUL', 'seat_number': 'T190074225', 'prn': '72139090M', 'result': {'310241': {'code': '310241', 'course_name': 'DATABASE MANAGEMENT SYSTEMS *', 'insem_obtain': 25, 'insem_total': 30, 'ese_obtain': 50, 'ese_total': 70, 'total_obtain': 75, 'total_total': 100, 'tw_obtain': None, 'tw_total': None, 'pr_obtain': None, 'pr_total': None, 'or_obtain': None, 'or_total': None, 'percent': 75, 'credits': 3, 'grade': 'A+', 'grade_points': 9.0, 'credit_points': 27.0, 'pass_or_remanded_status': None, 'order_of_appearance': None, 'semester': 'sem1'}, '310242': {'code': '310242', 'course_name': 'THEORY OF COMPUTATION *', 'insem_obtain': 16, 'insem_total': 30, 'ese_obtain': 33, 'ese_total': 70, 'total_obtain': 49, 'total_total': 100, 'tw_obtain': None, 'tw_total': None, 'pr_obtain': None, 'pr_total': None, 'or_obtain': None, 'or_total': None, 'percent': 49, 'credits': 3, 'grade': 'C', 'grade_points': 5.0, 'credit_points': 15.0, 'pass_or_remande